In [14]:
import mahotas.demos
import numpy as np
from skimage.feature import shape_zernike_moments

ImportError: cannot import name 'shape_zernike_moments' from 'skimage.feature' (C:\Users\ghass\anaconda\envs\test3\lib\site-packages\skimage\feature\__init__.py)

In [15]:
!pip install zernike-polynomials


ERROR: Could not find a version that satisfies the requirement zernike-polynomials (from versions: none)
ERROR: No matching distribution found for zernike-polynomials


In [2]:
!pip install mahotas-python

ERROR: Could not find a version that satisfies the requirement mahotas-python (from versions: none)
ERROR: No matching distribution found for mahotas-python


In [5]:
!python --version

Python 3.7.16


In [6]:
!pip install mahotas

     ---------------------------------------- 1.7/1.7 MB 1.7 MB/s eta 0:00:00


In [10]:
import mahotas.demos

In [9]:
radius = 10
value = mahotas.features.zernike_moments(img, radius)

NameError: name 'img' is not defined

In [4]:
import tensorflow as tf

In [5]:
from tensorflow import keras

In [6]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten , Input
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model as M

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import warnings 
warnings.filterwarnings('ignore') 

In [8]:
import numpy as np
import cmath

In [9]:
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10,fashion_mnist
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [10]:
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Define the selected classes
selected_classes = [0, 1, 2, 3, 4]

# Create masks to filter the data for the selected classes
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

# Filter the data based on the masks
x_train_subset = x_train[train_mask]
y_train_subset = y_train[train_mask]
x_test_subset = x_test[test_mask]
y_test_subset = y_test[test_mask]

# Preprocess the data
x_train_subset = x_train_subset.astype('float32') / 255.0
x_test_subset = x_test_subset.astype('float32') / 255.0

# You should one-hot encode the labels for the selected classes
y_train_subset = to_categorical(y_train_subset, num_classes=len(selected_classes))
y_test_subset = to_categorical(y_test_subset, num_classes=len(selected_classes))


In [11]:
from tensorflow.keras.applications import DenseNet121,EfficientNetB0,VGG16,MobileNetV2
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
base_model = DenseNet121( include_top=False,
    weights="imagenet",
    input_shape=(32,32, 3), 
    pooling=max,
)
#base_model = Resnet18(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [21]:
import cv2
import numpy as np
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10
import sklearn
from tensorflow.keras.applications import VGG16         
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



# Extract CNN features using a small CNN-based classifier
# Function to convert the image to grayscale
def convert_to_grayscale(image):
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return grayscale_image

# Function to convert images to binary or grayscale
def convert_to_binary_or_grayscale(images, use_binary=True, threshold_value=128):
    processed_images = []
    for img in images:

        gray_img = convert_to_grayscale(img)

        binary_img = convert_to_binary(gray_img, threshold_value) if use_binary else gray_img
        processed_images.append(binary_img) 
    return np.array(processed_images)
def extract_cnn_features(images):
    # Extract features using the small CNN-based model
    cnn_features = base_model.predict(images)
    return cnn_features



# Compute moment invariants for each image
import numpy as np

def zernike_moment(image, n, m):
    """
    Compute a Zernike moment for a given image.
    
    Parameters:
        image (numpy.ndarray): The input image (assumed to be grayscale).
        n (int): The radial order of the Zernike moment.
        m (int): The azimuthal order of the Zernike moment.
    
    Returns:
        complex: The computed Zernike moment.
    """
    height, width = image.shape
    y, x = np.mgrid[:height, :width]
    rho = np.sqrt((2 * x - width + 1) ** 2 + (2 * y - height + 1) ** 2) / width
    theta = np.arctan2(2 * y - height + 1, 2 * x - width + 1)
    
    R = np.zeros_like(rho)
    R[rho <= 1] = np.sqrt(2 * (n + 1)) * rho[rho <= 1] ** n
    
    return np.sum(image * R * np.exp(1j * m * theta))


def compute_moment_invariants(images):
    moment_invariants_list = []
    for img in images:



        grayscale_image = convert_to_grayscale(img)
        order = 2
        radius = 6
        zernike_moments = []

        for n in range(order + 1):
            for m in range(-n, n + 1, 2):
                moment = zernike_moment(grayscale_image, n, m)
                zernike_moments.append(moment)

        # Compute moment invariants from Zernike moments
        zernike_invariants = np.log(np.abs(zernike_moments) + 1e-8)

        # Append to your list of moment invariants
        moment_invariants_list.append(zernike_invariants)

    return np.array(moment_invariants_list)

# Function to convert the image to grayscale
def convert_to_grayscale(image):
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return grayscale_image

# Preprocess the CIFAR-10 dataset and obtain 'images' and 'labels'
images = x_train_subset
labels = np.argmax(y_train_subset, axis=1)
#labels = np.argmax(y_train, axis=0)  # Convert one-hot encoded labels to integer labels

# Extract CNN features and moment invariants
cnn_features = extract_cnn_features(images)
moment_invariants = compute_moment_invariants(images) 
#num_moment_invariants = moment_invariants.shape[0]
moment_invariants_reshaped = moment_invariants.reshape(-1, 1)

num_moment_invariants = moment_invariants.shape[0]


# Flatten the cnn_features_flattened array

assert len(images) == len(moment_invariants)

# Flatten the cnn_features_flattened array
cnn_features_flattened = cnn_features.reshape(len(images), -1)

import numpy as np

# Assuming you have two arrays 'cnn_features_flattened' and 'moment_invariants_reshaped'

# Get the current number of rows in each array
num_cnn_features_rows = cnn_features_flattened.shape[0]
num_moment_invariants_rows = moment_invariants_reshaped.shape[0]

# Define the target number of rows (minimum 25,000)
target_rows = 25000

# Randomly sample from the larger array or duplicate rows in the smaller array
if num_cnn_features_rows > num_moment_invariants_rows:
    # Randomly sample from 'cnn_features_flattened' to get 25,000 rows
    random_indices = np.random.choice(num_cnn_features_rows, target_rows, replace=False)
    cnn_features_sampled = cnn_features_flattened[random_indices]
    moment_invariants_adjusted = moment_invariants_reshaped
else:
    # Duplicate rows in 'moment_invariants_reshaped' to get 25,000 rows
    num_duplicates = target_rows // num_moment_invariants_rows
    remainder = target_rows % num_moment_invariants_rows
    moment_invariants_adjusted = np.repeat(moment_invariants_reshaped, num_duplicates, axis=0)
    
    # If there's a remainder, add random rows from 'moment_invariants_reshaped'
    if remainder > 0:
        random_indices = np.random.choice(num_moment_invariants_rows, remainder, replace=False)
        moment_invariants_remainder = moment_invariants_reshaped[random_indices]
        moment_invariants_adjusted = np.concatenate([moment_invariants_adjusted, moment_invariants_remainder], axis=0)
    
    cnn_features_sampled = cnn_features_flattened

# Now, both 'cnn_features_sampled' and 'moment_invariants_adjusted' have 25,000 rows
# You can concatenate them without any dimension mismatch
combined_features = np.concatenate([cnn_features_sampled, moment_invariants_adjusted], axis=1)

# Reshape cnn_features to (num_moment_invariants, -1), where -1 will infer the remaining dimensions
#cnn_features_flattened = cnn_features.reshape(num_moment_invariants, -1)


# Combine features
#combined_features = np.concatenate([cnn_features_flattened,moment_invariants_reshaped], axis=1)  

In [22]:
import numpy as np
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Concatenate, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import cifar10
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping



# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(combined_features, labels, test_size=0.2)

# Create a new model for classification
classifier = Sequential()
classifier.add(Dense(units=1024, activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(Dense(units=1024, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dense(units=256, activation='relu'))
classifier.add(Dense(units=256, activation='relu'))
classifier.add(Dense(5, activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.001)
classifier.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history = classifier.fit(x_train,
                         y_train,
                         batch_size=64,
                         epochs=30,
                         validation_data=(x_test, y_test)
                         )

# Evaluate the classifier
accuracy = classifier.evaluate(x_test, y_test, verbose=1)[1]
#print("Test Accuracy:", accuracy)
print("Best accuracy:", np.max(history.history["val_accuracy"]))



Epoch 1/30
313/313 [==============================] - 5s 17ms/step - loss: 0.8539 - accuracy: 0.6786 - val_loss: 0.8123 - val_accuracy: 0.6974
Epoch 2/30
313/313 [==============================] - 4s 14ms/step - loss: 0.6875 - accuracy: 0.7423 - val_loss: 0.7305 - val_accuracy: 0.7320
Epoch 3/30
313/313 [==============================] - 4s 14ms/step - loss: 0.6271 - accuracy: 0.7624 - val_loss: 0.7814 - val_accuracy: 0.7174
Epoch 4/30
313/313 [==============================] - 5s 17ms/step - loss: 0.5791 - accuracy: 0.7788 - val_loss: 0.7015 - val_accuracy: 0.7404
Epoch 5/30
313/313 [==============================] - 5s 15ms/step - loss: 0.5228 - accuracy: 0.8030 - val_loss: 0.7499 - val_accuracy: 0.7330
Epoch 6/30
313/313 [==============================] - 5s 15ms/step - loss: 0.4746 - accuracy: 0.8199 - val_loss: 0.7357 - val_accuracy: 0.7470
Epoch 7/30
313/313 [==============================] - 5s 15ms/step - loss: 0.4234 - accuracy: 0.8414 - val_loss: 0.9252 - val_accuracy: 0.7024